In [6]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import json
import seaborn as sns
from collections import Counter
import nltk
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

C:\Users\owltu\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\owltu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
from nltk.tokenize import sent_tokenize

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

In [9]:
yelp = pd.read_csv("yelp_balanced.csv")
yelp.head()

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,111089,S3r97tHMkzR4O-tiIIzwOQ,r0GLQAawvHXUd-xZow-fWQ,mzREMIknfmagJugibXrCsQ,4,2017-04-14,"Food is tasty, it's usually busy, but there al...",0,0,0
1,674606,lZTYcQXntUsLptcWCqyhmg,bDs5_ek9esUkgxFm1jFxfg,aZG9Hp8fdgzq37iuB1QTYg,1,2017-10-01,Probably one of the worst experiences I've eve...,1,0,0
2,3329480,sStohXbOEdPyrutK3zJSmg,mus1d8cMxuEJmP0zfV19hg,I4Nr-MVc26qWr08-S3Q1ow,4,2017-01-19,I was looking for a place to grab a quick bite...,0,0,0
3,3641542,Pvw7lbeyGGLVALvK0DwgDA,qrwH8z4nNUabqlsnwNoDCg,vAbRNF0hJt91oCUpm_sdKw,3,2014-01-09,"Meh, fish was a bit rubbery for $34 swordfish....",1,0,0
4,2054056,XWN8BVvLwcmgEW6xZuOKpg,cYvO88wNlRc_gd03caBZ7Q,sNVGdeOPeitJ3OWUQBINzQ,4,2012-02-29,I've heard so much about this place.. and I mu...,2,2,3


In [10]:
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return df

#yelp = clean_text(yelp, 'text', 'text_clean')

In [69]:
yelp_basic = yelp[['stars', 'text_clean']]
yelp_features = yelp[['stars', 'text', 'text_clean', 'useful', 'funny', 'cool']]

In [70]:
df_train_raw, df_test = train_test_split(yelp_features, test_size=0.2, 
                                         random_state=42, shuffle=True)
df_train_raw, df_dev = train_test_split(df_train_raw, test_size=0.2,
                                        random_state=42, shuffle=True)

df_train = df_train_raw

In [71]:
def extract_features(df_train, df_dev, df_test, df_extra):
    vectorizer = TfidfVectorizer(analyzer='word', 
                               stop_words='english',
                               ngram_range=(1, 2),
                               lowercase=True,
                               min_df=5,
                               binary=False)
    df_train["text_clean"] = df_train["text"][:]
    
    df_extra["text_clean_backup"] = df_extra["text_clean"].values
    import random
    for ix, review in enumerate(df_extra["text_clean"].values):
        r = review.split()
        for i in range(0, len(r)//5): #20 % of words
            r.pop(random.randrange(len(r)))
        df_extra["text_clean"].values[ix] = " ".join(r)
    df_train = clean_text(df_train, "text_clean", "text_clean")
    
    x_train = vectorizer.fit_transform(df_train.loc[:, 'text_clean'].values.astype('U'))
    x_dev = vectorizer.transform(df_dev.loc[:, 'text_clean'].values.astype('U'))
    x_test = vectorizer.transform(df_test.loc[:, 'text_clean'].values.astype('U'))
    y_train = df_train.loc[:, 'stars'].tolist()
    y_dev = df_dev.loc[:, 'stars'].tolist()
    y_test = df_test.loc[:, 'stars'].tolist()
    x_contra = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['contradictory'] == 1].values.astype('U'))
    y_contra = df_extra.loc[:, 'stars'][df_extra['contradictory'] == 1].tolist()
    
    x_figurative = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['figurative'] == 1].values.astype('U'))
    y_figurative = df_extra.loc[:, 'stars'][df_extra['figurative'] == 1].tolist()

    x_qual = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['qualifier'] == 1].values.astype('U'))
    y_qual = df_extra.loc[:, 'stars'][df_extra['qualifier'] == 1].tolist()

    
    return x_train, x_dev, x_test, x_contra, x_figurative, x_qual, y_train, y_dev, y_test, y_contra, y_figurative, y_qual

In [72]:
def short_extract(df_extra):
    vectorizer = TfidfVectorizer(analyzer='word', 
                               stop_words='english',
                               ngram_range=(1, 2),
                               lowercase=True,
                               min_df=5,
                               binary=False)
    df_extra["text_clean"] = df_extra["text"][:]
    df_extra["text_clean_backup"] = df_extra["text_clean"].values
    import random
    for ix, review in enumerate(df_extra["text_clean"].values):
        r = review.split()
        for i in range(0, len(r)//5): #20 % of words
            r.pop(random.randrange(len(r)))
        df_extra["text_clean"].values[ix] = " ".join(r)
    df_extra = clean_text(df_extra, "text_clean", "text_clean")
    
    x_train = vectorizer.fit_transform(df_train.loc[:, 'text_clean'].values.astype('U'))

    x_contra = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['contradictory'] == 1].values.astype('U'))
    y_contra = df_extra.loc[:, 'stars'][df_extra['contradictory'] == 1].tolist()
    
    x_figurative = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['figurative'] == 1].values.astype('U'))
    y_figurative = df_extra.loc[:, 'stars'][df_extra['figurative'] == 1].tolist()

    x_qual = vectorizer.transform(df_extra.loc[:, 'text_clean'][df_extra['qualifier'] == 1].values.astype('U'))
    y_qual = df_extra.loc[:, 'stars'][df_extra['qualifier'] == 1].tolist()

    
    return x_contra, x_figurative, x_qual, y_contra, y_figurative, y_qual

In [44]:
# useful = df_train.loc[:, 'useful'].tolist()
# funny = df_train.loc[:, 'funny'].tolist()
# cool = df_train.loc[:, 'cool'].tolist()

In [62]:
edges = pd.read_csv("example_sentences.csv")
#edges = clean_text(edges, 'text', 'text_clean')

In [64]:
edge_features = edges[['stars', 'text', 'useful', 'funny', 'cool', 'contradictory', 'figurative', 'confusing', 'qualifier']]

In [47]:
x_train, x_dev, x_test, x_contra, x_figurative, x_qual, y_train, y_dev, y_test, y_contra, y_figurative, y_qual = extract_features(df_train, df_dev, df_test, edge_features)

C:\Users\owltu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [48]:
#Original
def evaluate_model_Xy(model, X, y, y_pred=None, label="Training", model_name="model"):
    if y_pred is None:
        y_pred = model.predict(X)

    print(label + ' Set')
    print("Accuracy:", accuracy_score(y, y_pred))
    print()

    print(classification_report(y, y_pred, digits=4))

#     disp = plot_confusion_matrix(model, X, y, 
#                                cmap=plt.cm.Blues, normalize='true')
#     plt.savefig(model_name + "_" + label.lower() + ".eps")
#     plt.show()
#     print()
    
    return accuracy_score(y, y_pred)

In [49]:
def evaluate_model(model, model_name="model",
                   y_train_pred=None, y_dev_pred=None, y_test_pred=None):
    mod_training = evaluate_model_Xy(model, x_train, y_train, label="Training", model_name=model_name)
    mod_valid = evaluate_model_Xy(model, x_dev, y_dev, label="Validation", model_name=model_name)
    mod_test = evaluate_model_Xy(model, x_test, y_test, label="Testing", model_name=model_name)
    mod_contradict = evaluate_model_Xy(model, x_contra, y_contra, label="Contradictory", model_name=model_name)
    mod_figurative = evaluate_model_Xy(model, x_figurative, y_figurative, label="Figurative", model_name=model_name)
    mod_qual = evaluate_model_Xy(model, x_qual, y_qual, label="Qualifiers", model_name=model_name)

    return (mod_training, mod_valid, mod_test, mod_contradict, mod_figurative, mod_qual)

In [50]:
def short_evaluate(model, model_name="model",
                   y_train_pred=None, y_dev_pred=None, y_test_pred=None):

    mod_contradict = evaluate_model_Xy(model, x_contra, y_contra, label="Contradictory", model_name=model_name)
    mod_figurative = evaluate_model_Xy(model, x_figurative, y_figurative, label="Figurative", model_name=model_name)
    mod_qual = evaluate_model_Xy(model, x_qual, y_qual, label="Qualifiers", model_name=model_name)

    return (mod_contradict, mod_figurative, mod_qual)

#### Naive Bayes

In [51]:
clf_nb = MultinomialNB(alpha=0.5,
                       fit_prior=True)
clf_nb.fit(x_train, y_train)

MultinomialNB(alpha=0.5)

In [52]:
nb_model = evaluate_model(clf_nb, model_name="nb")

Training Set
Accuracy: 0.7736458333333334

              precision    recall  f1-score   support

           1     0.7202    0.8051    0.7603     19222
           2     0.7490    0.7084    0.7281     19184
           3     0.8019    0.7625    0.7817     19146
           4     0.7465    0.8270    0.7847     19259
           5     0.8724    0.7650    0.8152     19189

    accuracy                         0.7736     96000
   macro avg     0.7780    0.7736    0.7740     96000
weighted avg     0.7779    0.7736    0.7740     96000

Validation Set
Accuracy: 0.549625

              precision    recall  f1-score   support

           1     0.6220    0.6988    0.6582      4867
           2     0.4694    0.4373    0.4527      4782
           3     0.4689    0.4516    0.4601      4867
           4     0.4832    0.5764    0.5257      4724
           5     0.7400    0.5836    0.6526      4760

    accuracy                         0.5496     24000
   macro avg     0.5567    0.5495    0.5499     24000

C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### SVM

In [53]:
clf_sgd = make_pipeline(StandardScaler(with_mean=False),
                        SGDClassifier(loss='hinge',
                                      penalty='l2',
                                      alpha=30,
                                      max_iter=1000, 
                                      tol=1e-3,
                                      shuffle=True,
                                      verbose=1,
                                      n_jobs=-1,
                                      random_state=0,
                                      learning_rate='optimal',
                                      early_stopping=True,
                                      class_weight='balanced'))
clf_sgd.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1
-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1

Norm: 0.11, NNZs: 164543, Bias: -0.005905, T: 86400, Avg. loss: 0.517484
Total training time: 0.08 seconds.
Norm: 0.11, NNZs: 164962, Bias: -0.005723, T: 86400, Avg. loss: 0.529879
Total training time: 0.09 seconds.
Norm: 0.11, NNZs: 164700, Bias: -0.006037, T: 86400, Avg. loss: 0.472364
Total training time: 0.09 seconds.
Norm: 0.11, NNZs: 163998, Bias: -0.006264, T: 86400, Avg. loss: 0.461948
Total training time: 0.09 seconds.
Norm: 0.11, NNZs: 165174, Bias: -0.005459, T: 86400, Avg. loss: 0.528260
Total training time: 0.09 seconds.
-- Epoch 2
-- Epoch 2
-- Epoch 2
-- Epoch 2
-- Epoch 2
Norm: 0.11, NNZs: 164813, Bias: -0.006010, T: 172800, Avg. loss: 0.485552
Total training time: 0.23 seconds.
Norm: 0.11, NNZs: 165088, Bias: -0.005834, T: 172800, Avg. loss: 0.496397
Total training time: 0.25 seconds.
Norm: 0.11, NNZs: 164418, Bias: -0.006360, T: 172800, Avg. loss: 0.433614
Total training time: 0.25 seconds.
Norm: 0.11, NNZs: 16491

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.9s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s finished


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('sgdclassifier',
                 SGDClassifier(alpha=30, class_weight='balanced',
                               early_stopping=True, n_jobs=-1, random_state=0,
                               verbose=1))])

In [54]:
svm_mod = evaluate_model(clf_sgd, model_name="sgd")

Training Set
Accuracy: 0.8271458333333334

              precision    recall  f1-score   support

           1     0.7236    0.9351    0.8159     19222
           2     0.8801    0.7144    0.7886     19184
           3     0.8917    0.7574    0.8191     19146
           4     0.8349    0.8270    0.8309     19259
           5     0.8543    0.9015    0.8772     19189

    accuracy                         0.8271     96000
   macro avg     0.8369    0.8271    0.8264     96000
weighted avg     0.8368    0.8271    0.8264     96000

Validation Set
Accuracy: 0.536375

              precision    recall  f1-score   support

           1     0.5613    0.8050    0.6614      4867
           2     0.4818    0.3453    0.4022      4782
           3     0.4730    0.3403    0.3958      4867
           4     0.4676    0.5040    0.4851      4724
           5     0.6534    0.6863    0.6695      4760

    accuracy                         0.5364     24000
   macro avg     0.5274    0.5362    0.5228     24000

In [58]:
test = edge_features.copy(deep=True)
short_extract(test)

KeyError: 'text'

In [59]:
edges

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool,contradictory,figurative,confusing,qualifier,text_clean
0,479303,akaDLBO67jdpvGlkLlfubg,COllio9e_alx3f-qCV7ELg,7UFDAX4wLi6ux5otguYldA,2,9/19/2015,Service was incredible! Food was simple but no...,2,1,1,1.0,0.0,0.0,1.0,service was incredible food was simple but not...
1,920853,-26jAeakSNH0XJajAHuT_g,Wu-yg4k0EgOeU2xuG6tcyw,vFTKENEf3AxbodEshK5W2A,2,6/1/2017,"Food was OK, but this is a smoky local hole in...",0,0,0,1.0,0.0,0.0,0.0,food was ok but this is a smoky local hole in ...
2,2369177,QGttFsUNCK7C8Bf4NzuXUQ,2BBdoPu5ACwvSaEe6Xm-Ew,i6N0e1zVhkQvof_acn85Og,1,2/6/2016,"The food was decent, however, the wait for del...",0,1,0,1.0,0.0,0.0,0.0,the food was decent however the wait for deliv...
3,3450986,3gb5P4DXMLelCuYDxPIPJw,zJ5iNcvHPehUfBlqqb5O1w,DkIEhR8behaIoSexekWV0Q,2,4/8/2014,I wanted to grab something quick while we wait...,2,0,0,1.0,0.0,0.0,0.0,i wanted to grab something quick while we wait...
4,3929939,gJnzHdKmAFjUK4uSwm4mlw,wWZ6LhvkcCqiSQNtVmGFYw,GdPWJo3z4ySEXpF7Wkn3FA,1,2/6/2015,Worst service ever. Food is really good but wa...,0,0,0,1.0,0.0,0.0,1.0,worst service ever food is really good but wai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,4063427,c5HbzxeMAptfa7orZShwiA,TSlAhLK_mWLDj6yNQsPvug,nz8uRlnveku1C3t-Lripkw,2,12/11/2016,I love this establishment. Karaoke with great ...,1,1,0,0.0,0.0,0.0,0.0,i love this establishment karaoke with great k...
156,1627654,bX833Z6EdmhHPOnrkmFX3A,dM1HHK4szA5Qs6NSAaEk1Q,LR0qF0FEVsCOhYWUOiH26A,3,12/1/2014,I think I went to this buffet like three times...,1,1,2,0.0,0.0,1.0,0.0,i think i went to this buffet like three times...
157,241787,3HwzXkgrYEbm5A1r-MjE0w,XwgnVwRyXYMxT3tRsEelqQ,kp9o9uv_scnT6ZBdDvvLaA,2,12/1/2017,I take great pride in finding the positive. Th...,0,0,0,0.0,1.0,0.0,0.0,i take great pride in finding the positive the...
158,349066,HeWf4ZMtBUZT8BZIZEQ10w,xo8CqeZRkhwe_Je9_OU65w,KljAJzE4WuV1ay503jjP6w,1,12/31/2016,"Just like everyone else has written, good food...",0,0,0,1.0,0.0,1.0,0.0,just like everyone else has written good food ...


In [61]:
edge_features

,stars,text_clean,useful,funny,cool,contradictory,figurative,confusing,qualifier,text_clean_backup
0,2,incredible food was simple but not impressivet...,2,1,1,1.0,0.0,0.0,1.0,service was incredible food was simple but not...
1,2,was ok this is smoky local hole in the wall se...,0,0,0,1.0,0.0,0.0,0.0,food was ok but this is a smoky local hole in ...
2,1,the food was decent however the for delivery w...,0,1,0,1.0,0.0,0.0,0.0,the food was decent however the wait for deliv...
3,2,i wanted to grab something quick we to board o...,2,0,0,1.0,0.0,0.0,0.0,i wanted to grab something quick while we wait...
4,1,worst service food is really good but waiters ...,0,0,0,1.0,0.0,0.0,1.0,worst service ever food is really good but wai...
...,...,...,...,...,...,...,...,...,...,...
155,2,i love this establishment karaoke great every ...,1,1,0,0.0,0.0,0.0,0.0,i love this establishment karaoke with great k...
156,3,think i went to this buffet like three times i...,1,1,2,0.0,0.0,1.0,0.0,i think i went to this buffet like three times...
157,2,i take great pride in the positive the was exc...,0,0,0,0.0,1.0,0.0,0.0,i take great pride in finding the positive the...
158,1,just has written good food but customer chicke...,0,0,0,1.0,0.0,1.0,0.0,just like everyone else has written good food ...


In [77]:
niter = 30
edge_naive = []*niter
edge_svm = []*niter
for i in range(niter):
    x_contra, x_figurative, x_qual, y_contra, y_figurative, y_qual = short_extract(edge_features.copy(deep=True))   
    edge_naive.append(short_evaluate(clf_nb, model_name="nb"))
    edge_svm.append(short_evaluate(clf_sgd, model_name="sgd"))

Contradictory Set
Accuracy: 0.5048543689320388

              precision    recall  f1-score   support

           1     0.3000    0.3000    0.3000        10
           2     0.6111    0.5500    0.5789        40
           3     0.5455    0.4615    0.5000        39
           4     0.2632    0.5556    0.3571         9
           5     0.8000    0.8000    0.8000         5

    accuracy                         0.5049       103
   macro avg     0.5039    0.5334    0.5072       103
weighted avg     0.5348    0.5049    0.5133       103

Figurative Set
Accuracy: 0.5

              precision    recall  f1-score   support

           1     0.4000    0.3333    0.3636         6
           2     0.5714    0.4706    0.5161        17
           3     0.5000    0.4737    0.4865        19
           4     0.3333    0.6667    0.4444         6
           5     0.8000    0.6667    0.7273         6

    accuracy                         0.5000        54
   macro avg     0.5210    0.5222    0.5076        54

C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Contradictory Set
Accuracy: 0.5631067961165048

              precision    recall  f1-score   support

           1     0.6667    0.4000    0.5000        10
           2     0.6579    0.6250    0.6410        40
           3     0.5789    0.5641    0.5714        39
           4     0.3333    0.4444    0.3810         9
           5     0.3333    0.6000    0.4286         5

    accuracy                         0.5631       103
   macro avg     0.5140    0.5267    0.5044       103
weighted avg     0.5847    0.5631    0.5679       103

Figurative Set
Accuracy: 0.5370370370370371

              precision    recall  f1-score   support

           1     0.7143    0.8333    0.7692         6
           2     0.6429    0.5294    0.5806        17
           3     0.5625    0.4737    0.5143        19
           4     0.2000    0.3333    0.2500         6
           5     0.5714    0.6667    0.6154         6

    accuracy                         0.5370        54
   macro avg     0.5382    0.5673    0

C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Contradictory Set
Accuracy: 0.5145631067961165

              precision    recall  f1-score   support

           1     0.5000    0.5000    0.5000        10
           2     0.6216    0.5750    0.5974        40
           3     0.5000    0.4103    0.4507        39
           4     0.2778    0.5556    0.3704         9
           5     0.6667    0.8000    0.7273         5

    accuracy                         0.5146       103
   macro avg     0.5132    0.5682    0.5291       103
weighted avg     0.5359    0.5146    0.5189       103

Figurative Set
Accuracy: 0.5185185185185185

              precision    recall  f1-score   support

           1     0.5714    0.6667    0.6154         6
           2     0.6000    0.5294    0.5625        17
           3     0.6154    0.4211    0.5000        19
           4     0.2143    0.5000    0.3000         6
           5     0.8000    0.6667    0.7273         6

    accuracy                         0.5185        54
   macro avg     0.5602    0.5568    0

...

C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Contradictory Set
Accuracy: 0.47572815533980584

              precision    recall  f1-score   support

           1     0.3636    0.4000    0.3810        10
           2     0.6571    0.5750    0.6133        40
           3     0.4857    0.4359    0.4595        39
           4     0.1765    0.3333    0.2308         9
           5     0.4000    0.4000    0.4000         5

    accuracy                         0.4757       103
   macro avg     0.4166    0.4288    0.4169       103
weighted avg     0.5093    0.4757    0.4887       103

Figurative Set
Accuracy: 0.42592592592592593

              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000         6
           2     0.5294    0.5294    0.5294        17
           3     0.5714    0.4211    0.4848        19
           4     0.2308    0.5000    0.3158         6
           5     0.5000    0.5000    0.5000         6

    accuracy                         0.4259        54
   macro avg     0.3663    0.3901   

C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\owltu\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Contradictory Set
Accuracy: 0.5145631067961165

              precision    recall  f1-score   support

           1     0.4545    0.5000    0.4762        10
           2     0.6970    0.5750    0.6301        40
           3     0.5294    0.4615    0.4932        39
           4     0.2222    0.4444    0.2963         9
           5     0.4286    0.6000    0.5000         5

    accuracy                         0.5146       103
   macro avg     0.4663    0.5162    0.4792       103
weighted avg     0.5555    0.5146    0.5278       103

Figurative Set
Accuracy: 0.46296296296296297

              precision    recall  f1-score   support

           1     0.6000    0.5000    0.5455         6
           2     0.5000    0.4118    0.4516        17
           3     0.5556    0.5263    0.5405        19
           4     0.2500    0.5000    0.3333         6
           5     0.4000    0.3333    0.3636         6

    accuracy                         0.4630        54
   macro avg     0.4611    0.4543    

Contradictory Set
Accuracy: 0.5145631067961165

              precision    recall  f1-score   support

           1     0.3636    0.4000    0.3810        10
           2     0.6667    0.6000    0.6316        40
           3     0.5312    0.4359    0.4789        39
           4     0.2500    0.4444    0.3200         9
           5     0.5000    0.8000    0.6154         5

    accuracy                         0.5146       103
   macro avg     0.4623    0.5361    0.4854       103
weighted avg     0.5415    0.5146    0.5214       103

Figurative Set
Accuracy: 0.6111111111111112

              precision    recall  f1-score   support

           1     0.5000    0.6667    0.5714         6
           2     0.7647    0.7647    0.7647        17
           3     0.7692    0.5263    0.6250        19
           4     0.3333    0.5000    0.4000         6
           5     0.4286    0.5000    0.4615         6

    accuracy                         0.6111        54
   macro avg     0.5592    0.5915    0

In [78]:
edge_naive = np.array(edge_naive)
edge_svm = np.array(edge_svm)

In [79]:
edge_naive

array([[0.50485437, 0.5       , 0.51351351],
       [0.47572816, 0.44444444, 0.59459459],
       [0.5631068 , 0.53703704, 0.62162162],
       [0.51456311, 0.51851852, 0.48648649],
       [0.45631068, 0.44444444, 0.56756757],
       [0.52427184, 0.5       , 0.54054054],
       [0.50485437, 0.55555556, 0.62162162],
       [0.48543689, 0.46296296, 0.59459459],
       [0.54368932, 0.51851852, 0.56756757],
       [0.48543689, 0.53703704, 0.62162162],
       [0.48543689, 0.42592593, 0.56756757],
       [0.45631068, 0.33333333, 0.48648649],
       [0.51456311, 0.42592593, 0.64864865],
       [0.46601942, 0.48148148, 0.54054054],
       [0.53398058, 0.5       , 0.59459459],
       [0.58252427, 0.44444444, 0.59459459],
       [0.54368932, 0.48148148, 0.64864865],
       [0.4368932 , 0.46296296, 0.51351351],
       [0.53398058, 0.46296296, 0.54054054],
       [0.44660194, 0.48148148, 0.51351351],
       [0.53398058, 0.53703704, 0.62162162],
       [0.57281553, 0.48148148, 0.67567568],
       [0.

In [55]:
print(np.mean(edge_naive[:,0]), np.mean(edge_naive[:,1]), np.mean(edge_naive[:,2]))

0.5016181229773463 0.47098765432098766 0.555855855855856


In [56]:
print(np.mean(edge_svm[:,0]), np.mean(edge_svm[:,1]), np.mean(edge_svm[:,2]))

0.42038834951456305 0.5074074074074074 0.44054054054054054


In [81]:
from scipy import stats

In [61]:
stats.describe(edge_naive[:,1])

DescribeResult(nobs=30, minmax=(0.35185185185185186, 0.5555555555555556), mean=0.47098765432098766, variance=0.0024285353893697876, skewness=-0.15338630892875782, kurtosis=-0.27608008352196123)

In [82]:
print(stats.describe(edge_naive[:,0]), stats.describe(edge_naive[:,1]), stats.describe(edge_naive[:,2]), sep="\n")

DescribeResult(nobs=30, minmax=(0.4368932038834951, 0.5825242718446602), mean=0.509061488673139, variance=0.0013630803600933052, skewness=-0.08075407614592454, kurtosis=-0.6168101033575222)
DescribeResult(nobs=30, minmax=(0.3333333333333333, 0.6111111111111112), mean=0.48333333333333334, variance=0.002775412705170051, skewness=-0.18499191643450744, kurtosis=1.1660800786579193)
DescribeResult(nobs=30, minmax=(0.4594594594594595, 0.6756756756756757), mean=0.5738738738738739, variance=0.0030066413104623724, skewness=-0.17193341608718485, kurtosis=-0.7763261702424158)


In [83]:
print(stats.describe(edge_svm[:,0]), stats.describe(edge_svm[:,1]), stats.describe(edge_svm[:,2]), sep = "\n")

DescribeResult(nobs=30, minmax=(0.36893203883495146, 0.49514563106796117), mean=0.4359223300970874, variance=0.0011333903224653116, skewness=-0.07096092652368005, kurtosis=-0.7630878393087568)
DescribeResult(nobs=30, minmax=(0.4074074074074074, 0.5925925925925926), mean=0.5117283950617284, variance=0.001950790722608518, skewness=-0.08287045871298422, kurtosis=-0.20838562633670543)
DescribeResult(nobs=30, minmax=(0.3783783783783784, 0.5945945945945946), mean=0.45585585585585586, variance=0.002606147620127116, skewness=0.38229421538769337, kurtosis=-0.006735572324367389)
